In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

#We use headers to make the browser look like a user so you don't get blocked

res = requests.get("https://www.espn.com/nba/seasonleaders", headers=headers)
res.raise_for_status()

In [3]:
soup = BeautifulSoup(res.text, "lxml")

player_tbl = soup.find("table", {"class": "tablehead"})

raw_players = player_tbl.find_all("tr")
player_li = []
for player in raw_players:
  if "colhead" in player.get("class", []) or "stathead" in player.get("class", []):
    continue
  player_li.append(player)

In [ ]:
player_names = []
for player in player_li:
  for data in player.find_all("td"):
    name = data.find("a")
    if name:
      player_names.append(name.text)

Nikola Jokic
Giannis Antetokounmpo
Shai Gilgeous-Alexander
Luka Doncic
Victor Wembanyama
Anthony Davis
Cade Cunningham
LeBron James
Jayson Tatum
Karl-Anthony Towns
Domantas Sabonis
Kevin Durant
Trae Young
Zion Williamson
Anthony Edwards
Damian Lillard
James Harden
Jalen Brunson
Devin Booker
Tyrese Maxey
Paolo Banchero
LaMelo Ball
Stephen Curry
Joel Embiid
Kyrie Irving
Tyler Herro
De'Aaron Fox
Jalen Johnson
Tyrese Haliburton
Franz Wagner
Ja Morant
Ivica Zubac
Alperen Sengun
Donovan Mitchell
Jalen Williams
Nikola Vucevic
Evan Mobley
Jamal Murray
Scottie Barnes
Zach LaVine
Brandon Ingram
Bam Adebayo
Jaylen Brown
Kawhi Leonard
Desmond Bane
Pascal Siakam
Darius Garland
RJ Barrett
Austin Reaves
Jaren Jackson Jr.


### This section was used to extract the top 50 players according to the ESPN Season Leaders site for the 2024-2025 NBA season. We will be using these players to gather **stats** and **Points Asts props** to train our model to determine whether you should bet over or under on a certain day.